In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [22]:
import pandas as pd

# Define file paths
btc_price_path = "../datasets/normalised_bitcoin_price_hours.parquet"
sentiment_path = "../datasets/hourly_sentiment.parquet"

# Load datasets
btc_data = pd.read_parquet(btc_price_path)
sentiment_data = pd.read_parquet(sentiment_path)

# Convert 'date' columns to datetime format (if not already)
btc_data['datetime'] = pd.to_datetime(btc_data['datetime'])
sentiment_data['datetime'] = pd.to_datetime(sentiment_data['datetime'])

# Display loaded datasets
print("Bitcoin Price Data:")
print(btc_data.head())
print("\nSentiment Data:")
print(sentiment_data.head())

Bitcoin Price Data:
             datetime      Open      High       Low     Close    Volume
0 2021-01-01 00:00:00  0.250730  0.251106  0.250628  0.250493  0.076344
1 2021-01-01 01:00:00  0.251433  0.251695  0.251353  0.251520  0.015258
2 2021-01-01 02:00:00  0.256154  0.256011  0.255263  0.255877  0.034476
3 2021-01-01 03:00:00  0.252147  0.252928  0.252351  0.253123  0.010795
4 2021-01-01 04:00:00  0.254932  0.255284  0.254180  0.254066  0.023016

Sentiment Data:
             datetime  sentiment_score  bert_sentiment
0 2021-02-05 10:00:00         0.274245        0.611672
1 2021-02-05 11:00:00         0.110898        0.604489
2 2021-02-05 12:00:00         0.192854        0.606953
3 2021-02-05 13:00:00         0.142706        0.603910
4 2021-02-05 14:00:00         0.127426        0.607347


In [23]:
# Ensure both VADER and BERT sentiment scores are included in sentiment_data
if 'bert_sentiment' not in sentiment_data.columns:
    raise ValueError("BERT sentiment score column is missing in sentiment dataset!")

# Merge on 'date' column (inner join to keep common dates)
merged_data = pd.merge(btc_data, sentiment_data, on='datetime', how='inner')

# Display merged dataset
print("Merged Dataset with VADER & BERT Sentiments:")
print(merged_data[['datetime', 'Close', 'sentiment_score', 'bert_sentiment']].head())

Merged Dataset with VADER & BERT Sentiments:
             datetime     Close  sentiment_score  bert_sentiment
0 2021-02-05 10:00:00  0.410087         0.274245        0.611672
1 2021-02-05 11:00:00  0.405529         0.110898        0.604489
2 2021-02-05 12:00:00  0.412469         0.192854        0.606953
3 2021-02-05 13:00:00  0.417710         0.142706        0.603910
4 2021-02-05 14:00:00  0.417754         0.127426        0.607347


In [24]:
# Merge on 'date' column (left join to keep all dates from btc_data)
merged_data = pd.merge(btc_data, sentiment_data, on='datetime', how='left')

# Fill missing values with a default value (e.g., 0 for sentiment scores)
merged_data = merged_data.copy()  # Ensure it is a proper DataFrame copy
merged_data['sentiment_score'] = merged_data['sentiment_score'].fillna(0)
merged_data['bert_sentiment'].fillna(0, inplace=True)

# Display merged dataset
print("Merged Dataset with Left Join and Filled Missing Values:")
print(merged_data[['datetime', 'Close', 'sentiment_score', 'bert_sentiment']].head())

Merged Dataset with Left Join and Filled Missing Values:
             datetime     Close  sentiment_score  bert_sentiment
0 2021-01-01 00:00:00  0.250493              0.0             0.0
1 2021-01-01 01:00:00  0.251520              0.0             0.0
2 2021-01-01 02:00:00  0.255877              0.0             0.0
3 2021-01-01 03:00:00  0.253123              0.0             0.0
4 2021-01-01 04:00:00  0.254066              0.0             0.0


/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_84459/1862007621.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_data['bert_sentiment'].fillna(0, inplace=True)


In [20]:
# Create lag features (Previous day's Close price, VADER sentiment, BERT sentiment)
merged_data['prev_close'] = merged_data['Close'].shift(1)
merged_data['prev_vader_sentiment'] = merged_data['sentiment_score'].shift(1)
merged_data['prev_bert_sentiment'] = merged_data['bert_sentiment'].shift(1)

# Display dataset with lag features
print("Dataset with Lag Features:")
print(merged_data[['datetime', 'Close', 'prev_close', 'sentiment_score', 'bert_sentiment', 'prev_vader_sentiment', 'prev_bert_sentiment']].head())

Dataset with Lag Features:
             datetime     Close  prev_close  sentiment_score  bert_sentiment  \
0 2021-01-01 00:00:00  0.250493         NaN              0.0             0.0   
1 2021-01-01 01:00:00  0.251520    0.250493              0.0             0.0   
2 2021-01-01 02:00:00  0.255877    0.251520              0.0             0.0   
3 2021-01-01 03:00:00  0.253123    0.255877              0.0             0.0   
4 2021-01-01 04:00:00  0.254066    0.253123              0.0             0.0   

   prev_vader_sentiment  prev_bert_sentiment  
0                   NaN                  NaN  
1                   0.0                  0.0  
2                   0.0                  0.0  
3                   0.0                  0.0  
4                   0.0                  0.0  


In [25]:
# Create lag features (Previous day's Close price, VADER sentiment, BERT sentiment)
merged_data['prev_close'] = merged_data['Close'].shift(1)
merged_data['prev_vader_sentiment'] = merged_data['sentiment_score'].shift(1)
merged_data['prev_bert_sentiment'] = merged_data['bert_sentiment'].shift(1)

# Display dataset with lag features
print("Dataset with Lag Features:")
print(merged_data[['datetime', 'Close', 'prev_close', 'sentiment_score', 'bert_sentiment', 'prev_vader_sentiment', 'prev_bert_sentiment']].head())

Dataset with Lag Features:
             datetime     Close  prev_close  sentiment_score  bert_sentiment  \
0 2021-01-01 00:00:00  0.250493         NaN              0.0             0.0   
1 2021-01-01 01:00:00  0.251520    0.250493              0.0             0.0   
2 2021-01-01 02:00:00  0.255877    0.251520              0.0             0.0   
3 2021-01-01 03:00:00  0.253123    0.255877              0.0             0.0   
4 2021-01-01 04:00:00  0.254066    0.253123              0.0             0.0   

   prev_vader_sentiment  prev_bert_sentiment  
0                   NaN                  NaN  
1                   0.0                  0.0  
2                   0.0                  0.0  
3                   0.0                  0.0  
4                   0.0                  0.0  


In [26]:
# Volatility indicators (Standard Deviation) for Hourly Data
merged_data['volatility_7d'] = merged_data['Close'].rolling(window=168).std()    # 7 days × 24 hours
merged_data['volatility_14d'] = merged_data['Close'].rolling(window=336).std()   # 14 days × 24 hours
merged_data['volatility_30d'] = merged_data['Close'].rolling(window=720).std()   # 30 days × 24 hours

# Display dataset with volatility indicators
print("Dataset with Volatility Indicators:")
print(merged_data[['datetime', 'Close', 'volatility_7d', 'volatility_14d', 'volatility_30d']].head(15))


Dataset with Volatility Indicators:
              datetime     Close  volatility_7d  volatility_14d  \
0  2021-01-01 00:00:00  0.250493            NaN             NaN   
1  2021-01-01 01:00:00  0.251520            NaN             NaN   
2  2021-01-01 02:00:00  0.255877            NaN             NaN   
3  2021-01-01 03:00:00  0.253123            NaN             NaN   
4  2021-01-01 04:00:00  0.254066            NaN             NaN   
5  2021-01-01 05:00:00  0.252553            NaN             NaN   
6  2021-01-01 06:00:00  0.253321            NaN             NaN   
7  2021-01-01 07:00:00  0.262223            NaN             NaN   
8  2021-01-01 08:00:00  0.257684            NaN             NaN   
9  2021-01-01 09:00:00  0.259726            NaN             NaN   
10 2021-01-01 10:00:00  0.257684            NaN             NaN   
11 2021-01-01 11:00:00  0.257372            NaN             NaN   
12 2021-01-01 12:00:00  0.256492            NaN             NaN   
13 2021-01-01 13:00:00  0.

In [27]:
# Calculate 7-day and 30-day moving averages of the closing prices for Hourly Data
merged_data['closing_7d_avg'] = merged_data['Close'].rolling(window=168).mean()   # 7 days × 24 hours
merged_data['closing_30d_avg'] = merged_data['Close'].rolling(window=720).mean()  # 30 days × 24 hours

# Display dataset with moving averages
print("Dataset with 7-day and 30-day Moving Averages:")
print(merged_data[['datetime', 'Close', 'closing_7d_avg', 'closing_30d_avg']].head(15))


Dataset with 7-day and 30-day Moving Averages:
              datetime     Close  closing_7d_avg  closing_30d_avg
0  2021-01-01 00:00:00  0.250493             NaN              NaN
1  2021-01-01 01:00:00  0.251520             NaN              NaN
2  2021-01-01 02:00:00  0.255877             NaN              NaN
3  2021-01-01 03:00:00  0.253123             NaN              NaN
4  2021-01-01 04:00:00  0.254066             NaN              NaN
5  2021-01-01 05:00:00  0.252553             NaN              NaN
6  2021-01-01 06:00:00  0.253321             NaN              NaN
7  2021-01-01 07:00:00  0.262223             NaN              NaN
8  2021-01-01 08:00:00  0.257684             NaN              NaN
9  2021-01-01 09:00:00  0.259726             NaN              NaN
10 2021-01-01 10:00:00  0.257684             NaN              NaN
11 2021-01-01 11:00:00  0.257372             NaN              NaN
12 2021-01-01 12:00:00  0.256492             NaN              NaN
13 2021-01-01 13:00:00  0.256

In [28]:
# Handling missing values: Forward Fill (then Backward Fill if needed)
merged_data.fillna(method='ffill', inplace=True)
merged_data.fillna(method='bfill', inplace=True)

# Display final dataset
print("Final Dataset After Handling Missing Values:")
merged_data.head(15)

Final Dataset After Handling Missing Values:


/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_84459/3130139557.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data.fillna(method='ffill', inplace=True)
/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_84459/3130139557.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data.fillna(method='bfill', inplace=True)


,datetime,Open,High,Low,Close,Volume,sentiment_score,bert_sentiment,prev_close,prev_vader_sentiment,prev_bert_sentiment,volatility_7d,volatility_14d,volatility_30d,closing_7d_avg,closing_30d_avg
0,2021-01-01 00:00:00,0.250730,0.251106,0.250628,0.250493,0.076344,0.0,0.0,0.250493,0.0,0.0,0.050683,0.068865,0.057312,0.320975,0.357583
1,2021-01-01 01:00:00,0.251433,0.251695,0.251353,0.251520,0.015258,0.0,0.0,0.250493,0.0,0.0,0.050683,0.068865,0.057312,0.320975,0.357583
2,2021-01-01 02:00:00,0.256154,0.256011,0.255263,0.255877,0.034476,0.0,0.0,0.251520,0.0,0.0,0.050683,0.068865,0.057312,0.320975,0.357583
3,2021-01-01 03:00:00,0.252147,0.252928,0.252351,0.253123,0.010795,0.0,0.0,0.255877,0.0,0.0,0.050683,0.068865,0.057312,0.320975,0.357583
4,2021-01-01 04:00:00,0.254932,0.255284,0.254180,0.254066,0.023016,0.0,0.0,0.253123,0.0,0.0,0.050683,0.068865,0.057312,0.320975,0.357583
5,2021-01-01 05:00:00,0.252374,0.252653,0.252569,0.252553,0.004519,0.0,0.0,0.254066,0.0,0.0,0.050683,0.068865,0.057312,0.320975,0.357583
6,2021-01-01 06:00:00,0.253714,0.253685,0.253472,0.253321,0.035770,0.0,0.0,0.252553,0.0,0.0,0.050683,0.068865,0.057312,0.320975,0.357583
7,2021-01-01 07:00:00,0.261518,0.262348,0.261729,0.262223,0.074643,0.0,0.0,0.253321,0.0,0.0,0.050683,0.068865,0.057312,0.320975,0.357583
8,2021-01-01 08:00:00,0.257084,0.257484,0.257103,0.257684,0.012742,0.0,0.0,0.262223,0.0,0.0,0.050683,0.068865,0.057312,0.320975,0.357583
9,2021-01-01 09:00:00,0.259018,0.259780,0.259227,0.259726,0.026793,0.0,0.0,0.257684,0.0,0.0,0.050683,0.068865,0.057312,0.320975,0.357583


In [29]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18265 entries, 0 to 18264
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   datetime              18265 non-null  datetime64[ns]
 1   Open                  18265 non-null  float64       
 2   High                  18265 non-null  float64       
 3   Low                   18265 non-null  float64       
 4   Close                 18265 non-null  float64       
 5   Volume                18265 non-null  float64       
 6   sentiment_score       18265 non-null  float64       
 7   bert_sentiment        18265 non-null  float64       
 8   prev_close            18265 non-null  float64       
 9   prev_vader_sentiment  18265 non-null  float64       
 10  prev_bert_sentiment   18265 non-null  float64       
 11  volatility_7d         18265 non-null  float64       
 12  volatility_14d        18265 non-null  float64       
 13  volatility_30d  

In [30]:
# Save the final processed dataset
final_dataset_path = "../datasets/final_merged_dataset_hourly.parquet"
merged_data.to_parquet(final_dataset_path, index=False)

print(f"Processed dataset saved at: {final_dataset_path}")

Processed dataset saved at: ../datasets/final_merged_dataset_hourly.parquet


In [31]:
merged_data.shape

(18265, 16)